In [3]:
import torch

# Simulated predictions and targets (replace with actual data)
predictions = torch.tensor([0, 1, 2, 0, 2, 1])
targets = torch.tensor([0, 1, 2, 1, 2, 0])

# Calculate the multiclass confusion matrix
confusion_matrix = torch.zeros(3, 3)

for t, p in zip(targets, predictions):
    confusion_matrix[t, p] += 1

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix)

Confusion Matrix:
tensor([[1., 1., 0.],
        [1., 1., 0.],
        [0., 0., 2.]])
